Enhanced record minimum standard (ERMS) is the minimum standard of data enhancement for heritage places. The report of Heritage Places ERMS is done downstream, once the heritages places (HP) have been recorded in the database

## Install

## Python libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import ipywidgets as widgets
from IPython.display import display
import plotly.express as px
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex

### Environment

Clone [eamena-functions](https://github.com/eamena-project/eamena-functions) repo and import the script `erms.py`

ℹ️ Do not run locally

In [2]:
!rm /content/eamena-functions -R
!git clone https://github.com/eamena-project/eamena-functions.git
%cd /content/eamena-functions
from erms import erms

Cloning into 'eamena-functions'...
remote: Enumerating objects: 187, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 187 (delta 108), reused 120 (delta 46), pack-reused 0
Receiving objects: 100% (187/187), 46.13 KiB | 1.77 MiB/s, done.
Resolving deltas: 100% (108/108), done.
/content/eamena-functions


## Reference Data

Show the dataframe where the ERMS field are listes (`Yes` in the column `Enhanced record minimum standard`)

In [3]:
erms_template = erms.erms_template()
erms_template

,level1,level2,level3,Enhanced record minimum standard
0,ASSESSMENT SUMMARY,ASSESSMENT ACTIVITY,Assessment Investigator - Actor,NaN
1,ASSESSMENT SUMMARY,ASSESSMENT ACTIVITY,Investigator Role Type,Yes
2,ASSESSMENT SUMMARY,ASSESSMENT ACTIVITY,Assessment Activity Type,NaN
3,ASSESSMENT SUMMARY,ASSESSMENT ACTIVITY,Assessment Activity Date,NaN
4,ASSESSMENT SUMMARY,ASSESSMENT ACTIVITY,Google Earth Assessment,NaN
...,...,...,...,...
93,ENVIRONMENT ASSESSMENT,DEPTH or ELEVATION,Minimum Depth/Max Elevation,NaN
94,ENVIRONMENT ASSESSMENT,DEPTH or ELEVATION,Maximum Depth/Min Elevation,NaN
95,ENVIRONMENT ASSESSMENT,DEPTH or ELEVATION,Datum Type,NaN
96,ENVIRONMENT ASSESSMENT,DEPTH or ELEVATION,Datum Description/EPSG code,NaN


## Select the level of aggregation

See the documentation for

In [4]:
options=['level1', 'level2', 'level3']
radio_button = widgets.RadioButtons(
    options=options,
    description='Select an option:'
)
display(radio_button)

RadioButtons(description='Select an option:', options=('level1', 'level2', 'level3'), value='level1')

Show the aggregated ERMS dataframe. The column `value` has `1` for any given fields beloging to the ERMS standards

In [5]:
df_erms = erms.erms_template_levels(radio_button = radio_button)
print(df_erms.to_markdown(index=False))

You selected: level3
| field                                              |   value |
|:---------------------------------------------------|--------:|
| AH Date From                                       |       0 |
| AH Date To                                         |       0 |
| Address                                            |       0 |
| Address Type                                       |       0 |
| Administrative Division                            |       0 |
| Administrative Division Type                       |       0 |
| Archaeological From Date                           |       0 |
| Archaeological To Date                             |       0 |
| Assessment Activity Date                           |       0 |
| Assessment Activity Type                           |       0 |
| Assessment Investigator - Actor                    |       0 |
| BP Date From                                       |       0 |
| BP Date To                                         |       0 |
| Be

### Read

In [6]:
erms_template = erms.erms_template()
erms_template = erms_template.drop('Enhanced record minimum standard', axis=1)
erms_template

,level1,level2,level3
0,ASSESSMENT SUMMARY,ASSESSMENT ACTIVITY,Assessment Investigator - Actor
1,ASSESSMENT SUMMARY,ASSESSMENT ACTIVITY,Investigator Role Type
2,ASSESSMENT SUMMARY,ASSESSMENT ACTIVITY,Assessment Activity Type
3,ASSESSMENT SUMMARY,ASSESSMENT ACTIVITY,Assessment Activity Date
4,ASSESSMENT SUMMARY,ASSESSMENT ACTIVITY,Google Earth Assessment
...,...,...,...
93,ENVIRONMENT ASSESSMENT,DEPTH or ELEVATION,Minimum Depth/Max Elevation
94,ENVIRONMENT ASSESSMENT,DEPTH or ELEVATION,Maximum Depth/Min Elevation
95,ENVIRONMENT ASSESSMENT,DEPTH or ELEVATION,Datum Type
96,ENVIRONMENT ASSESSMENT,DEPTH or ELEVATION,Datum Description/EPSG code


In [7]:
# import kaleido
# pio.orca.config.executable = '/usr/local/bin/orca'
def color_fields(level = 'level1'):
  erms_template = erms.erms_template()
  erms_template = erms_template.drop('Enhanced record minimum standard', axis=1)
  level1_cat = erms_template[level].unique()
  my_cmap = plt.get_cmap('Dark2')
  level1_cmap = my_cmap(np.linspace(0, 1, len(level1_cat)))
  level1_cmap = [to_hex(color) for color in level1_cmap]
  df_color = pd.DataFrame(columns=(level, 'color'))
  df_color[level] = level1_cat
  df_color['color'] = level1_cmap
  df_color = erms_template.merge(df_color, on=level, how='left')
  return df_color
df_color = color_fields()

df_color
# level1
erms_template = erms.erms_template()
my_cmap = plt.get_cmap('Dark2')
df_level1 = erms_template['level1'].value_counts()
df_level1 = df_level1.reset_index()
df_level1.columns = ['level1', 'value']
level1_cmap = my_cmap(np.linspace(0, 1, len(df_level1)))
level1_cmap = [to_hex(color) for color in level1_cmap]
df_level1['color'] = level1_cmap
df_level1['color_fill'] = df_level1['color'] + '55' # '#ffffffff'
#sort to keep the original order
sorter_level1 = list(erms_template['level1'].unique())
df_level1.sort_values(by="level1", key=lambda column: column.map(lambda e: sorter_level1.index(e)), inplace=True)
df_level1
# - - - - - - - - - - -
# level3
df_color = color_fields()
df_color3 = df_color.drop(columns=['level2'])
df_color3.drop_duplicates(inplace = True, keep='first')
# level3
erms_template = erms.erms_template()
df_level3 = erms_template['level3'].value_counts()
df_level3 = df_level3.reset_index()
df_level3.columns = ['level3', 'value']
df_level3 = df_level3.merge(df_color3, on='level3', how='left')
#sort to keep the original order
sorter_level3 = list(erms_template['level3'].unique())
df_level3.sort_values(by="level3", key=lambda column: column.map(lambda e: sorter_level3.index(e)), inplace=True)
# df_level3['lbl'] = str(range(len(df_level3)+1)) + "." + df_level3['level3']
df_level3['lbl'] = range(len(df_level3))
df_level3['lbl'] = df_level3['lbl'] + 1
df_level3['lbl'] = df_level3['lbl'].astype("string")
df_level3['lbl'] = "f." + df_level3['lbl']
df_level3['legend'] = df_level3['lbl'] + ": " + df_level3['level3']
df_erms_level3 = df_erms.rename(columns={'field': 'level3'})
df_level3_template = df_level3.drop('value', axis=1)
df_erms_level3 = df_erms_level3.merge(df_level3_template, on='level3', how='left')
# white when not within ERMS
df_erms_level3.loc[df_erms_level3['value'] == 0, 'color'] = df_erms_level3.loc[df_erms_level3['value'] == 0, 'color'] + '55' # '#ffffffff'
df_erms_level3['value'] = 1
# # keep order
sorter_level3 = list(erms_template['level3'].unique())
df_erms_level3.sort_values(by="level3", key=lambda column: column.map(lambda e: sorter_level3.index(e)), inplace=True)
df_erms_level3
# df_level1 = df_erms_level3['level1'].value_counts()
# df_level1 = df_level1.reset_index()
# df_level1.columns = ['level1', 'value']
# level1_cmap = my_cmap(np.linspace(0, 1, len(df_level1)))
# level1_cmap = [to_hex(color) for color in level1_cmap]
# df_level1['color'] = level1_cmap
# df_level1['color_fill'] = df_level1['color'] + '00' # '#ffffffff'
# #sort to keep the original order
# sorter_level1 = list(erms_template['level1'].unique())
# df_level1.sort_values(by="level1", key=lambda column: column.map(lambda e: sorter_level1.index(e)), inplace=True)
# df_level1
# - - -
fig_erms_level = go.Figure()
fig_erms_level.add_trace(go.Pie(
        # top
        labels=df_erms_level3['level3'],
        values=df_erms_level3['value'],
        sort=False,
        marker=dict(colors=df_erms_level3['color'], line=dict(color='#000000', width=0)),
        # hoverinfo='none',
        hoverinfo='skip',
        textinfo='label',
        textposition='inside',
        textfont_size=10,
        customdata=df_erms_level3["level3"],
        hovertemplate='<br>'.join(['%{customdata}<extra></extra>']),
        showlegend=False,
        domain={'x':[0.2,0.8], 'y':[0.1,0.9]},
        hole=0.1,
        # hovertemplate='%{x}, %{y}<extra></extra>'
        # legendgroup='legend'
        ))
# fig_erms_level.update_layout(hovermode=False)
# level1
fig_erms_level.add_trace(go.Pie(
        labels=df_level1['level1'],
        values=df_level1['value'],
        sort=False,
        # opacity=0.20,
        # mode="markers",
        marker=dict(colors=level1_cmap, line=dict(color='#000000', width=0)), # df_level1['color']
        # textinfo=None,
        textinfo='label',
        textfont_size=14,
        textposition='inside',
        customdata=df_level1["level1"],
        hovertemplate='<br>'.join(['%{customdata}<extra></extra>']),
        legendgroup = '1',
        rotation= -26,
        opacity = .55,
        # direction= "clockwise",
        domain={'x':[0.1,0.9], 'y':[0,1]},
        hole=0.8,
        showlegend=False,
        # hovertemplate='%{customdata}<extra></extra>'
        ))
fig_erms_level.update_layout(
    title_text="EAMENA Heritage Places fields<br><sup>ERMS fields are highlighted</sup>",
    title_x=0.5,
    autosize=False,
    width=1000,
    height=1000,
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='fields', x=0.5, y=0.5, font_size=10, showarrow=False)])
fig_erms_level.show()


# fig_erms_level.write_image("image.png")
# from IPython.display import Image
# Image("image.png")

# fig_erms_level.write_image("yourfile.png")
# fig_erms_level.write_image('/content/figure.png')  # Adjust the file path as needed
from google.colab import files
go.Figure.write_html(fig_erms_level,"erms_level.html") # write as html or image
files.download("erms_level.html") # download your file and give me a vote my answer
# # fig_level1.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>